In [6]:
import pandas
import pandas as pd
import sqlite3

DATA_PATH = "../Data/"

# Load data to sqlite

In [7]:
posts = pd.read_csv(DATA_PATH + "Posts.csv.gz", compression = 'gzip')
comments = pd.read_csv(DATA_PATH + "Comments.csv.gz", compression = 'gzip')
users = pd.read_csv(DATA_PATH + "Users.csv.gz", compression = 'gzip')


In [8]:
db = "database.db"
conn = sqlite3.connect(db)
try:
    comments.to_sql("Comments", conn)
    posts.to_sql("Posts", conn)
    users.to_sql("Users", conn)
except ValueError:
    print("Data already loaded")

Data already loaded


# Create comparator func

In [9]:
def comparator(df1,df2):
    assert type(df1) == pd.DataFrame
    assert type(df2) == pd.DataFrame
    eq = df1.equals(df2)
    if eq:
        print("Dataframes are equal")
    else:
        print("Dataframes are not equal")

# Task 1

In [5]:
query = '''
SELECT Location, SUM(UpVotes) as TotalUpVotes
FROM Users
WHERE Location != ''
GROUP BY Location
ORDER BY TotalUpVotes DESC
LIMIT 10
'''
sql_res = pd.read_sql_query(query, conn)

In [6]:
users_non_null_location = users[users['Location'] != '']
p_res_unsorted = users_non_null_location.groupby('Location')['UpVotes'].sum()
p_res = p_res_unsorted.sort_values(ascending=False).head(10).to_frame().reset_index()
p_res.rename(columns={'UpVotes': 'TotalUpVotes'}, inplace=True)

In [7]:
comparator(sql_res, p_res)

Dataframes are equal


# Task 2

In [8]:
query = '''
SELECT STRFTIME('%Y', CreationDate) AS Year, STRFTIME('%m', CreationDate) AS Month,
COUNT(*) AS PostsNumber, MAX(Score) AS MaxScore
FROM Posts
WHERE PostTypeId IN (1, 2)
GROUP BY Year, Month
HAVING PostsNumber > 1000
'''
sql_res = pd.read_sql_query(query, conn)

In [9]:
filtered_posts = posts.loc[posts['PostTypeId'].isin([1, 2])].copy()
filtered_posts['Year'] = filtered_posts['CreationDate'].apply(lambda x: x[:4])
filtered_posts['Month'] = filtered_posts['CreationDate'].apply(lambda x: x[5:7])
grouped = filtered_posts.groupby(['Year', 'Month']).agg(
    PostsNumber=('Id', 'count'), MaxScore=('Score', 'max'))

p_res = grouped[grouped['PostsNumber'] > 1000].reset_index()

In [10]:
comparator(sql_res, p_res)

Dataframes are equal


# Task 3

In [ ]:
query = '''
SELECT Id, DisplayName, TotalViews
FROM (
SELECT OwnerUserId, SUM(ViewCount) as TotalViews
FROM Posts
WHERE PostTypeId = 1
GROUP BY OwnerUserId
) AS Questions
JOIN Users
ON Users.Id = Questions.OwnerUserId
ORDER BY TotalViews DESC
LIMIT 10
'''
sql_res = pd.read_sql_query(query, conn)

In [29]:
questions = posts[posts["PostTypeId"] == 1].groupby("OwnerUserId").agg({"ViewCount": "sum"}).reset_index()
questions.columns = ["Id", "TotalViews"]

merged = users.merge(questions, on="Id").sort_values("TotalViews", ascending=False).head(10)
p_res = merged[["Id", "DisplayName", "TotalViews"]]
p_res.reset_index(drop=True, inplace=True)

In [31]:
comparator(sql_res, p_res)

Dataframes are equal


# Task 4

In [8]:
query = '''
SELECT DisplayName, QuestionsNumber, AnswersNumber, Location, Reputation, UpVotes, DownVotes
FROM (
SELECT *
FROM (
SELECT COUNT(*) as AnswersNumber, OwnerUserId
FROM Posts
WHERE PostTypeId = 2
GROUP BY OwnerUserId
) AS Answers
JOIN
(
SELECT COUNT(*) as QuestionsNumber, OwnerUserId
FROM Posts
WHERE PostTypeId = 1
GROUP BY OwnerUserId
) AS Questions
ON Answers.OwnerUserId = Questions.OwnerUserId
WHERE AnswersNumber > QuestionsNumber
ORDER BY AnswersNumber DESC
LIMIT 5
) AS PostsCounts
JOIN Users
ON PostsCounts.OwnerUserId = Users.Id
'''
sql_res = pd.read_sql_query(query, conn)

In [24]:
answers = posts[posts["PostTypeId"] == 2].groupby("OwnerUserId").agg(AnswersNumber=("Id", "count")).reset_index()
answers.columns = ["Id", "AnswersNumber"]
questions = posts[posts["PostTypeId"] == 1].groupby("OwnerUserId").agg(QuestionsNumber=("Id", "count")).reset_index()
questions.columns = ["Id", "QuestionsNumber"]
user_posts_counts = answers.merge(questions, on='Id')
filtered_users = user_posts_counts[user_posts_counts['AnswersNumber'] > user_posts_counts['QuestionsNumber']]
top_users = filtered_users.sort_values(by='AnswersNumber', ascending=False).head(5)
top_users_data = top_users.merge(users, on='Id')
result = top_users_data[['DisplayName', 'QuestionsNumber', 'AnswersNumber', 'Location', 'Reputation', 'UpVotes', 'DownVotes']]

In [26]:
comparator(sql_res, result)

Dataframes are equal


# Task 5

In [ ]:
query = '''
SELECT Title, CommentCount, ViewCount, CommentsTotalScore, DisplayName, Reputation, Location
FROM (
SELECT Posts.OwnerUserId, Posts.Title, Posts.CommentCount, Posts.ViewCount,
CmtTotScr.CommentsTotalScore
FROM (
SELECT PostId, SUM(Score) AS CommentsTotalScore
FROM Comments
GROUP BY PostId
) AS CmtTotScr
JOIN Posts ON Posts.Id = CmtTotScr.PostId
WHERE Posts.PostTypeId=1
) AS PostsBestComments
JOIN Users ON PostsBestComments.OwnerUserId = Users.Id
ORDER BY CommentsTotalScore DESC
LIMIT 10
'''
sql_res = pd.read_sql_query(query, conn)

In [17]:
cmt_tot_scr = comments.groupby('PostId')['Score'].sum().reset_index()
cmt_tot_scr.columns = ['PostId', 'CommentsTotalScore']
posts_best_comments = cmt_tot_scr.merge(posts[posts['PostTypeId'] == 1], left_on='PostId', right_on='Id')
posts_best_comments = posts_best_comments[['OwnerUserId', 'Title', 'CommentCount', 'ViewCount', 'CommentsTotalScore']]
result = posts_best_comments.merge(users, left_on='OwnerUserId', right_on='Id')
result = result[['Title', 'CommentCount', 'ViewCount', 'CommentsTotalScore', 'DisplayName', 'Reputation', 'Location']]

result_limit = result.sort_values(by='CommentsTotalScore', ascending=False).head(10)
result_limit = result_limit.reset_index(drop=True)

In [ ]:
comparator(sql_res, result_limit)